# YOLO Real-time Object Detection 

Using a pretrained YOLO model, this algorithm can segment objects ( cars, bicycles, buses..) and people in a given image.
This algorithm has multiple use cases: 
- Autonomous cars 
- Autonomous trucks
- Autonomous ships
- Collision avoidance 

Part 3 contains all the references for this model. Several functions are from Allan Zelener - YAD2K: Yet Another Darknet 2 Keras (https://github.com/allanzelener/YAD2K)

Below are the packages and dependencies.

In [ ]:
import argparse
import os
from os import listdir
from os.path import isfile, join
from keras import backend as K
from keras.layers import Input, Lambda, Conv2D
from keras.models import load_model, Model
from yad2k.utils.yolo_utils import read_classes, read_anchors, generate_colors, preprocess_image, draw_boxes, scale_boxes, detect
from yad2k.models.keras_yolo import yolo_head, yolo_boxes_to_corners, preprocess_true_boxes, yolo_loss, yolo_body, yolo_filter_boxes, yolo_eval

%matplotlib inline

## 1 - YOLO model details

YOLO (You Only Look Once) is an accurate real-time Object detection alogrithm where you only look once at an image to perform dectection. The algorithm below can detect 60 object classes listed felow 

<img src="nb_images/table 1.JPG" style="width:700px;height:300;">
<caption><center> <u> **Figure 1 </u>:** List of object classes <br> </center></caption>


The model relies on 3 steps: 
- Resize the image to the appropriate input size
- Run a Deep convolutional neural network
- Filter anchor boxes using score thresholding and intersection over union thresholding     

<img src="nb_images/figure1.JPG" style="width:900px;height:400;">
<caption><center> <u> **Figure 2 </u>:** Overall model summary<br> </center></caption>

## 2 - Pretrained model

In this part, we are going to use a pretrained model. 

### 2.1 - Defining classes, anchors and image shape.

In the cell below, we create the session and define the classes and anchors. We will also input image size.

In [ ]:
sess = K.get_session()
class_names = read_classes("model_data/coco_classes.txt")
anchors = read_anchors("model_data/yolo_anchors.txt")
image_shape = (1224., 1632.)    

### 2.2 - Loading a pretrained model

We are going to load an existing pretrained Keras YOLO model whose weights are stored in "yolo.h5", which can be downloaded from the official Yolo website

In [ ]:
yolo_model = load_model("model_data/yolo.h5")

Here are the layers the model contains:

In [ ]:
yolo_model.summary()

### 2.3 - Convert output of the model to usable bounding box tensors

The output of `yolo_model` is a (m, 19, 19, 5, 85) tensor that needs to pass through non-trivial processing and conversion. The following cell does that for you.

In [ ]:
yolo_outputs = yolo_head(yolo_model.output, anchors, len(class_names))

You added `yolo_outputs` to your graph. This set of 4 tensors is ready to be used as input by your `yolo_eval` function.

### 2.4 - Filtering boxes

`yolo_outputs` gave you all the predicted boxes of `yolo_model` in the correct format. You're now ready to perform filtering and select only the best boxes. Lets now call `yolo_eval`, which you had previously implemented, to do this. 

In [ ]:
boxes, scores, classes = yolo_eval(yolo_outputs, image_shape)

### 2.5 - Test on images

Input images will be placed in images/ directory. The detect function will perform the Object detection 

Run the following cell on the "test.jpg" image to verify that your function is correct.

In [ ]:
out_scores, out_boxes, out_classes = detect(sess, "test1.jpg", scores, boxes, classes, yolo_model, class_names)

Run on all files in images/ directory and save it to output/ directory. Images must have same size

In [ ]:
all_images = [detect(sess, img, scores, boxes, classes, yolo_model, class_names) for img in listdir('images/') if isfile(join('images/', img))]

## 3 - References

- Allan Zelener - YAD2K: Yet Another Darknet 2 Keras (https://github.com/allanzelener/YAD2K)
- Joseph Redmon, Santosh Divvala, Ross Girshick, Ali Farhadi - You Only Look Once: Unified, Real-Time Object Detection (2015)
- Joseph Redmon, Ali Farhadi - YOLO9000: Better, Faster, Stronger (2016)
- YOLO website (https://pjreddie.com/darknet/yolo/)